In [3]:
from simulator.bb84 import simulation_bb84
from tqdm import tqdm
from numpy import arange

In [5]:
results_list = []
for qber in tqdm(arange(start=0.01, stop=0.16, step = 0.01)):
    simulation_results = simulation_bb84(
        alice_basis_length=10000,
        disturbance_probability=qber,
        cascade_n_passes=4
    )
    results_list.append(simulation_results)
    # How long will it take to perform this simple test?
    # TODO: handling of no error estimation (like in the '93 paper)

 20%|██        | 3/15 [00:04<00:16,  1.39s/it]

[[34, 68, 136, 272], 10000, 1.0, 0.04, 0.02421858955928862, 3961, 0.2548363208770752, 0.5, 0.2, 4, 'AttributeError for binary_previous']


NameError: name 'exit' is not defined